# Data preprocessing

Computational Identification of Protein Phosphorylation Sites

recommand running on Google Colab (cd-hit only support linux environment)

Imports

In [1]:
import pandas as pd
import subprocess
import os

### Read data

In [3]:
file_path = './train.txt' # upload file to Colab before running
data = pd.read_csv(file_path, sep='\t', header=None)

data.columns = ['ID', 'UniProt_ID', 'Position', 'Modification', 'Protein', 'Species', 'Sequence', 'Source', 'PubMed_ID']

data['Original_Index'] = data.index

data

,ID,UniProt_ID,Position,Modification,Protein,Species,Sequence,Source,PubMed_ID,Original_Index
0,CPLM000002,O00116,108,Sumoylation,AGPS,Homo sapiens,MAEAAAAAGGTGLGAGASYGSAADRDRDPDPDRAGRRLRVLSGHLL...,Exp.,28112733,0
1,CPLM000002,O00116,169,Sumoylation,AGPS,Homo sapiens,MAEAAAAAGGTGLGAGASYGSAADRDRDPDPDRAGRRLRVLSGHLL...,Exp.,28112733,1
2,CPLM000002,O00116,424,Sumoylation,AGPS,Homo sapiens,MAEAAAAAGGTGLGAGASYGSAADRDRDPDPDRAGRRLRVLSGHLL...,Exp.,28112733,2
3,CPLM000002,O00116,456,Sumoylation,AGPS,Homo sapiens,MAEAAAAAGGTGLGAGASYGSAADRDRDPDPDRAGRRLRVLSGHLL...,Exp.,28112733,3
4,CPLM000005,O00139-1,119,Sumoylation,KIF2A,Homo sapiens,MVTSLNEDNESVTVEWIENGDTKGKEIDLESIFSLNPDLVPDEEIE...,Exp.,28112733,4
...,...,...,...,...,...,...,...,...,...,...
23643,CPLM036285,Q60817,127,Sumoylation,Naca,Mus musculus,MPGEATETVPATEQELPQPQAETGSGTESDSDESVPELEEQDSTQT...,Dat.,31014809,23643
23644,CPLM036321,Q60974,1330,Sumoylation,Ncor1,Mus musculus,MSSSGYPPNQGAFSTEQSRYPSHSVQYTFPSARHQQEFAVPDYRSS...,Dat.,3755602,23644
23645,CPLM036321,Q60974,1117,Sumoylation,Ncor1,Mus musculus,MSSSGYPPNQGAFSTEQSRYPSHSVQYTFPSARHQQEFAVPDYRSS...,Dat.,3755601,23645
23646,CPLM036321,Q60974,152,Sumoylation,Ncor1,Mus musculus,MSSSGYPPNQGAFSTEQSRYPSHSVQYTFPSARHQQEFAVPDYRSS...,Exp./Dat.,16421255;3755600,23646


### Remove duplicate sequences and record positions

In [4]:
modification_positions = data.groupby('UniProt_ID').agg(
    Positive_Positions=('Position', list),   # Collect all positions
    Sequence=('Sequence', 'first'), # Take the first sequence as the representative
    Original_Index=('Original_Index', 'min') # Take the first index as the representative
).reset_index()

# Sort by the original index
modification_positions = modification_positions.sort_values('Original_Index').reset_index(drop=True)

# Sort the positions
modification_positions['Positive_Positions'] = modification_positions['Positive_Positions'].apply(sorted)

modification_positions

,UniProt_ID,Positive_Positions,Sequence,Original_Index
0,O00116,"[108, 169, 424, 456]",MAEAAAAAGGTGLGAGASYGSAADRDRDPDPDRAGRRLRVLSGHLL...,0
1,O00139-1,[119],MVTSLNEDNESVTVEWIENGDTKGKEIDLESIFSLNPDLVPDEEIE...,4
2,O00139-2,"[75, 345, 359, 437, 650]",MVTSLNEDNESVTVEWIENGDTKGKEIDLESIFSLNPDLVPDEEIE...,5
3,O00141,"[152, 317, 367]",MTVKTEAAKGTLTYSRMRGMVAILIAFMKQRRMGLNDFIQKIANNS...,10
4,O00148,"[31, 35, 52, 154, 155, 162, 187, 240, 255, 267...",MAEQDVENDLLDYDEEEEPQAPQESTPAPPKKDIKGSYVSIHSSGF...,13
...,...,...,...,...
3995,Q60765,[42],MMLQHPGQVSASEVSATAIVPCLSPPGSLVFEDFANLTPFVKEELR...,23641
3996,Q60793,[275],MRQPPGESDMAVSDALLPSFSTFASGPAGREKTLRPAGAPTNRWRE...,23642
3997,Q60817,[127],MPGEATETVPATEQELPQPQAETGSGTESDSDESVPELEEQDSTQT...,23643
3998,Q60974,"[152, 1117, 1330]",MSSSGYPPNQGAFSTEQSRYPSHSVQYTFPSARHQQEFAVPDYRSS...,23644


### Extract positive and negative data

In [5]:
def get_pos_and_neg_data(df, l_size, r_size = None):
    if r_size is None:
        r_size = l_size

    sequences = df['Sequence']

    count_pos = 0
    count_neg = 0
    if not ('Positive_Data' in df.columns):  df.insert(len(df.columns), 'Positive_Data', list)
    if not ('Negative_Data' in df.columns): df.insert(len(df.columns), 'Negative_Data', list)
    if not ('Negative_Positions' in df.columns): df.insert(len(df.columns), 'Negative_Positions', list)

    for idx, sequence in enumerate(sequences):
        pos_data = []
        neg_data = []
        pos_positions = df['Positive_Positions'][idx]
        neg_positions = []

        for pos in range(len(sequence)):
            if sequence[pos] == 'K':
                pos += 1 # number in positions is 1-based

                ## Positive data
                if pos in pos_positions:
                    start = pos - l_size - 1
                    end = pos + r_size

                    ### Out of bounds
                    if start < 0 or end > len(sequence):
                        continue

                    pos_data.append(sequence[start:end])
                    count_pos += 1

                ## Negative data
                else:
                    start = pos - l_size - 1
                    end = pos + r_size

                    ### Out of bounds
                    if start < 0 or end > len(sequence):
                        continue

                    ### Overlapping with positive data
                    l_bound = max(0, start - l_size - 1)
                    r_bound = min(len(sequence), end + r_size + 1)
                    if any([p_pos >= l_bound and p_pos <= r_bound or p_pos >= start and p_pos <= end for p_pos in pos_positions]):
                        continue

                    neg_data.append(sequence[start:end])
                    neg_positions.append(pos)
                    count_neg += 1

        df.at[idx, 'Positive_Data'] = pos_data
        df.at[idx, 'Negative_Data'] = neg_data
        df.at[idx, 'Negative_Positions'] = neg_positions


    print(f'Total Positive data: {count_pos}')
    print(f'Total Negative data: {count_neg}')

    return df


window_size = 5 # modify this to change the window size
modification_positions = get_pos_and_neg_data(modification_positions, window_size)


result = modification_positions[['UniProt_ID', 'Positive_Positions', 'Positive_Data', 'Negative_Positions', 'Negative_Data']]
result

Total Positive data: 23260
Total Negative data: 126946


,UniProt_ID,Positive_Positions,Positive_Data,Negative_Positions,Negative_Data
0,O00116,"[108, 169, 424, 456]","[FIFNKKGQIEL, FLHDLKETNIS, RLMDNKQFQFG, YITKFK...","[59, 85, 86, 92, 129, 143, 147, 220, 227, 292,...","[STNECKARRAA, SGTIPKKRQEV, GTIPKKRQEVM, RQEVMK..."
1,O00139-1,[119],[PVQAAKKEFGP],"[23, 25, 60, 63, 66, 75, 134, 138, 142, 145, 1...","[ENGDTKGKEID, GDTKGKEIDLE, PASSAKVNKIV, SAKVNK..."
2,O00139-2,"[75, 345, 359, 437, 650]","[TVASIKNDPPS, VLEDGKQQVQV, QEREVKCVEDV, GAEINK...","[23, 25, 60, 63, 110, 115, 119, 123, 126, 137,...","[ENGDTKGKEID, GDTKGKEIDLE, PASSAKVNKIV, SAKVNK..."
3,O00141,"[152, 317, 367]","[RNVLLKNVKHP, KPLQLKPNITN, DLINKKITPPF]","[9, 29, 41, 50, 59, 94, 102, 106, 111, 118, 12...","[KTEAAKGTLTY, LIAFMKQRRMG, NDFIQKIANNS, NSYACK..."
4,O00148,"[31, 35, 52, 154, 155, 162, 187, 240, 255, 267...","[TPAPPKKDIKG, PKKDIKGSYVS, RDFLLKPELLR, GGLSIK...","[89, 94, 130, 137, 221, 294, 333]","[VLCQAKSGMGK, KSGMGKTAVFV, AFQISKEYERF, YERFSK..."
...,...,...,...,...,...
3995,Q60765,[42],[LTPFVKEELRF],"[78, 89, 97, 102, 106, 107, 108, 110, 116, 120...","[EMSVTKSEAAP, EEDERKRRRRE, RRERNKIAAAK, KIAAAK..."
3996,Q60793,[275],[MCPKIKQEAVP],"[32, 52, 225, 229, 249, 384, 386, 395, 409, 41...","[PAGREKTLRPA, ELSHMKRLPPL, GGLMGKFVLKA, GKFVLK..."
3997,Q60817,[127],[VFGEAKIEDLS],"[66, 68, 74, 75, 78, 82, 98, 100, 108, 113, 14...","[EEPVSKAKQSR, PVSKAKQSRSE, QSRSEKKARKA, SRSEKK..."
3998,Q60974,"[152, 1117, 1330]","[PAFGVKHEAPS, PLTYIKQEEFS, YPKQIKRESPP]","[107, 175, 178, 194, 201, 203, 204, 205, 215, ...","[DSLESKRPRLE, NASPSKLSKEE, PSKLSKEELIQ, DREIAK..."


### Create coresponding folders

In [6]:
kmer_folder = f'./%dmer/' % (2 * window_size + 1)
sequence_folder = kmer_folder + 'sequence/'
cd_hit_folder = kmer_folder + 'cd_hit/'
os.mkdir(kmer_folder)
os.mkdir(sequence_folder)
os.mkdir(cd_hit_folder)

### Generate the FASTA files

In [7]:
def save_FASTA(df, file_path, data_title, position_title):
    with open(file_path, 'w') as file:
        for idx in range(len(df)):
            data = df[data_title][idx]
            positions = df[position_title][idx]
            ID = df['UniProt_ID'][idx]

            for i, fragment in enumerate(data):
                file.write(f'>{ID}|{positions[i]}\n')
                file.write(f'{fragment}\n')


save_FASTA(result, sequence_folder + 'positive_sequences.fasta', 'Positive_Data', 'Positive_Positions')
save_FASTA(result, sequence_folder + 'negative_sequences.fasta', 'Negative_Data', 'Negative_Positions')

### Install CD-HIT

In [8]:
!apt-get update

!apt-get install cd-hit

!which cd-hit

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,326 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,387 kB]
Get:14 http://security.

### Generate CD-HIT FASTA files

| Sequence Identity Threshold (`-c`) | Word Size (`-n`) |
|------------------|-------------|
| 100% (`-c 1.0`)  | `-n 5`      |
| 95%  (`-c 0.95`) | `-n 5`      |
| 90%  (`-c 0.9`)  | `-n 5`      |
| 85%  (`-c 0.85`) | `-n 5`      |
| 80%  (`-c 0.8`)  | `-n 5`      |
| 75%  (`-c 0.75`) | `-n 5`      |
| 70%  (`-c 0.7`)  | `-n 4`      |
| 60%  (`-c 0.6`)  | `-n 3`      |
| 50%  (`-c 0.5`)  | `-n 2`      |
| 40%  (`-c 0.4`)  | `-n 2`      |

In [9]:
pos_in = sequence_folder + 'positive_sequences.fasta'
neg_in = sequence_folder + 'negative_sequences.fasta'

# CD-HIT configs table
t = [
  [1.0, 5], [0.95, 5], [0.9, 5], [0.85, 5], [0.8, 5], [0.75, 5], [0.7, 4], [0.6, 3], [0.5, 2], [0.4, 2],
]

pos_clstr_records = []
neg_clstr_records = []

for i, _ in enumerate(t):
  # Variables
  threshold = t[i][0]
  word_size = t[i][1]
  pos_out = cd_hit_folder + f'positive_cd_hit{int(threshold*100)}.fasta'
  neg_out = cd_hit_folder + f'negative_cd_hit{int(threshold*100)}.fasta'

  # To str
  threshold = str(threshold)
  word_size = str(word_size)

  # Run CD-HIT
  err = subprocess.run([f'cd-hit -i {pos_in} -o {pos_out} -c {threshold} -n {word_size}'], shell=True, capture_output=True, text=True).stderr
  if err != None: print(err)
  err = subprocess.run([f'cd-hit -i {neg_in} -o {neg_out} -c {threshold} -n {word_size}'], shell=True, capture_output=True, text=True).stderr
  if err != None: print(err)

  # Count clustr number
  pos_result = subprocess.run([f'cat {pos_out} | wc -l'], capture_output=True, text=True, shell=True).stdout
  neg_result = subprocess.run([f'cat {neg_out} | wc -l'], capture_output=True, text=True, shell=True).stdout

  # Record
  pos_clstr = int(int(pos_result) / 2)
  neg_clstr = int(int(neg_result) / 2)
  pos_clstr_records.append(pos_clstr)
  neg_clstr_records.append(neg_clstr)

  print(f'threshold: {threshold} completed with pos: {pos_clstr}, neg: {neg_clstr}')



threshold: 1.0 completed with pos: 21076, neg: 107297


threshold: 0.95 completed with pos: 21076, neg: 107297


threshold: 0.9 completed with pos: 20383, neg: 97514


threshold: 0.85 completed with pos: 20313, neg: 96948


threshold: 0.8 completed with pos: 19434, neg: 87421


threshold: 0.75 completed with pos: 19399, neg: 87233


threshold: 0.7 completed with pos: 18349, neg: 77092


threshold: 0.6 completed with pos: 14121, neg: 46085


threshold: 0.5 completed with pos: 5377, neg: 11018


threshold: 0.4 completed with pos: 1253, neg: 1952


### Build the table

In [10]:
print('Sequence identity|Positive data|Negative data')
for i, _ in enumerate(t):
  threshold = int(t[i][0] * 100)
  print('%16d%s|%13d|%13d' % (threshold, '%', pos_clstr_records[i], neg_clstr_records[i]))

Sequence identity|Positive data|Negative data
             100%|        21076|       107297
              95%|        21076|       107297
              90%|        20383|        97514
              85%|        20313|        96948
              80%|        19434|        87421
              75%|        19399|        87233
              70%|        18349|        77092
              60%|        14121|        46085
              50%|         5377|        11018
              40%|         1253|         1952


### Download folder from Colab

In [14]:
zip_file = f'./{2 * window_size + 1}mer.zip'
subprocess.run([f'zip -r {zip_file} {kmer_folder}'], shell=True)
from google.colab import files
files.download(zip_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>